## Experiment 2 LLaMA3.2: Image-to-Text Matching

Set-up

In [ ]:
from huggingface_hub import login
import torch

login()

HF_TOKEN = "your_huggingface_token_here"  # Replace with your Hugging Face token

# This will be removed in the final version - LLaMA requires access from Meta
hf_key = HF_TOKEN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [2]:
import torch.nn.functional as F
import random
import pandas as pd
import os
from PIL import Image
from transformers import AutoProcessor, MllamaForConditionalGeneration
from tqdm import tqdm

Label selection

In [3]:
possible_labels_s_r = [
    "looloo", "loolah", "looloh", "loonoo", "loonah", "loonoh", "loomoo", "loomah", "loomoh",
    "lahloo", "lahlah", "lahloh", "lahnoo", "lahnah", "lahnoh", "lahmoo", "lahmah", "lahmoh",
    "lohloo", "lohlah", "lohloh", "lohnoo", "lohnah", "lohnoh", "lohmoo", "lohmah", "lohmoh",
    "nooloo", "noolah", "nooloh", "noonoo", "noonah", "noonoh", "noomoo", "noomah", "noomoh",
    "nahloo", "nahlah", "nahloh", "nahnoo", "nahnah", "nahnoh", "nahmoo", "nahmah", "nahmoh",
    "nohloo", "nohlah", "nohloh", "nohnoo", "nohnah", "nohnoh", "nohmoo", "nohmah", "nohmoh",
    "mooloo", "moolah", "mooloh", "moonoo", "moonah", "moonoh", "moomoo", "moomah", "moomoh",
    "mahloo", "mahlah", "mahloh", "mahnoo", "mahnah", "mahnoh", "mahmoo", "mahmah", "mahmoh",
    "mohloo", "mohlah", "mohloh", "mohnoo", "mohnah", "mohnoh", "mohmoo", "mohmah", "mohmoh"
]

possible_labels_p_nr = [
    "teetee", "teetuh", "teetay", "teekee", "teekuh", "teekay", "teepee", "teepuh", "teepay",
    "tuhtee", "tuhtuh", "tuhtay", "tuhkee", "tuhkuh", "tuhkay", "tuhpee", "tuhpuh", "tuhpay",
    "taytee", "taytuh", "taytay", "taykee", "taykuh", "taykay", "taypee", "taypuh", "taypay",
    "keetee", "keetuh", "keetay", "keekee", "keekuh", "keekay", "keepee", "keepuh", "keepay",
    "kuhtee", "kuhtuh", "kuhtay", "kuhkee", "kuhkuh", "kuhkay", "kuhpee", "kuhpuh", "kuhpay",
    "kaytee", "kaytuh", "kaytay", "kaykee", "kaykuh", "kaykay", "kaypee", "kaypuh", "kaypay",
    "peetee", "peetuh", "peetay", "peekee", "peekuh", "peekay", "peepee", "peepuh", "peepay",
    "puhtee", "puhtuh", "puhtay", "puhkee", "puhkuh", "puhkay", "puhpee", "puhpuh", "puhpay",
    "paytee", "paytuh", "paytay", "paykee", "paykuh", "paykay", "paypee", "paypuh", "paypay"
]

possible_labels_s_r = random.sample(possible_labels_s_r, 10)
possible_labels_p_nr = random.sample(possible_labels_p_nr, 10)

final_list = possible_labels_s_r + possible_labels_p_nr

print(final_list)

['mahnoo', 'lahlah', 'loonah', 'moomoo', 'nooloo', 'noonoh', 'moolah', 'lahmoo', 'lohmah', 'mahlah', 'tuhkuh', 'kuhkuh', 'teekuh', 'paytee', 'taytuh', 'peepay', 'puhkuh', 'tuhkay', 'puhpee', 'taypuh']


In [ ]:
class ImageTextMatcher:
    def __init__(self, image_folder="images"):
        """
        Initialize the analyzer with the path to the image folder.
        
        Args:
            image_folder: Path to the folder containing images
        """
        self.image_folder = image_folder
        self.results = []
        self.models = {}
        
    def load_model(self, model_name):
        """
        Load a multimodal model.
        
        Args:
            model_name: Name of the model to load ('llama')
        """
        if model_name == 'llama':
            processor = AutoProcessor.from_pretrained("meta-llama/Llama-3.2-11B-Vision-Instruct")
            model = MllamaForConditionalGeneration.from_pretrained(
                "meta-llama/Llama-3.2-11B-Vision-Instruct",
                torch_dtype=torch.float16,
                device_map="auto"
            )
            self.models['llama'] = {'model': model, 'processor': processor}
        else:
            raise ValueError(f"Unsupported model: {model_name}, Model not Found.")

    def classify_image(self, image_path, possible_labels):
        """
        Classify the image based on possible labels and calculate confidence score.
        
        Args:
            image_path: Path to the image file
            possible_labels: List of possible labels (S-R or P-NR)
            
        Returns:
            Predicted label and confidence score
        """
        model_info = self.models['llama']
        model = model_info['model']
        processor = model_info['processor']

        image = Image.open(image_path).convert('RGB')
        
        # Prompt setup
        messages = [
            {"role": "user", "content": [
                {"type": "image"},
                {"type": "text", "text": f"You are given an image for which you need to assign a label. Use one of the following labels: {possible_labels}. Only respond with the label."}
            ]}
        ]

        input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

        inputs = processor(
            image,
            input_text,
            add_special_tokens=False,
            return_tensors="pt"
        ).to(model.device)
        
        # Generate classification
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=5,
                output_scores=True,
                return_dict_in_generate=True,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                top_k=40
            )

            generated_token_ids = outputs.sequences[0][inputs['input_ids'].size(1):] # Only new tokens
            scores = outputs.scores # Logits for each new token

            token_probs = []
            for i, token_id in enumerate(generated_token_ids):
                logits = scores[i] # Logits for i-th token
                probs = F.softmax(logits, dim=-1)
                token_prob = probs[0, token_id]
                token_probs.append(token_prob.item())

            # Final predicted text (after decoding tokens)
            predicted_text = processor.tokenizer.decode(generated_token_ids, skip_special_tokens=True).strip()

            # Confidence score (mean of token probabilities)
            confidence_score = sum(token_probs) / len(token_probs)

            print(f"Predicted text: {predicted_text}")
            print(f"Token probabilities: {[round(p, 4) for p in token_probs]}")
            print(f"Mean confidence: {confidence_score:.4f}")

            return predicted_text, confidence_score

    def prepare_dataset_for_classification(self, image_paths=None):
        """
        Prepare dataset for image classification.
        
        Args:
            image_paths: List of paths to images (if None, scan the image folder)
            
        Returns:
            List of image paths
        """
        # If image paths are not provided, scan the image folder
        if image_paths is None:
            image_paths = []
            for filename in os.listdir(self.image_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    image_paths.append(os.path.join(self.image_folder, filename))
        
        return image_paths
    
    def classify_dataset(self, model_name, image_paths, possible_labels_s_r, possible_labels_p_nr):
        """
        Classify a dataset of images using S-R and P-NR labels and calculate confidence scores.
        
        Args:
            model_name: Name of the model to use
            image_paths: List of image paths to classify
            possible_labels_s_r: Labels for sonorant+rounded pseudowords
            possible_labels_p_nr: Labels for plosief+non-rounded pseudowords
            
        Returns:
            DataFrame with classification results
        """
        if model_name not in self.models:
            self.load_model(model_name)
        
        classification_results = []

        # Classify images in batches of 4
        batch_size = 4
        for i in range(0, len(image_paths), batch_size):
            batch = image_paths[i:i+batch_size]
            
            for image_path in tqdm(batch, desc=f"Classifying images {i}-{i+len(batch)}"):
                print("batch number: ", i)
                try:
                    # First classify with S-R labels
                    predicted_class_s_r, score_s_r = self.classify_image(image_path, possible_labels_s_r)
                    
                    # Then classify with P-NR labels
                    predicted_class_p_nr, score_p_nr = self.classify_image(image_path, possible_labels_p_nr)

                    filename = os.path.basename(image_path)
                    image_type = 'Unknown'
                    if 'curved' in filename.lower():
                        image_type = 'Curved'
                    elif 'jagged' in filename.lower():
                        image_type = 'Jagged'
                                
                    # Store result with confidence scores
                    classification_results.append({
                        'image_path': image_path,
                        'image_filename': os.path.basename(image_path),
                        'image_type': image_type,
                        'predicted_class_s_r': predicted_class_s_r,
                        'score_s_r': score_s_r,
                        'predicted_class_p_nr': predicted_class_p_nr,
                        'score_p_nr': score_p_nr
                    })
                    
                except Exception as e:
                    print(f"Error classifying {image_path}: {str(e)}")
                    continue

            # Clear CUDA cache between batches
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        return pd.DataFrame(classification_results)

    def analyze_classification_results(self, results_df):
        """
        Analyze the classification results.
        
        Args:
            results_df: DataFrame with classification results
            
        Returns:
            DataFrame with classification metrics
        """
        # Extract and analyze the scores for S-R and P-NR words
        curved_scores_s_r = results_df[results_df['image_type'] == 'Curved']['score_s_r']
        jagged_scores_s_r = results_df[results_df['image_type'] == 'Jagged']['score_s_r']
        
        curved_scores_p_nr = results_df[results_df['image_type'] == 'Curved']['score_p_nr']
        jagged_scores_p_nr = results_df[results_df['image_type'] == 'Jagged']['score_p_nr']
        
        # Compare average scores
        analysis_results = {
            'avg_score_s_r_curved': curved_scores_s_r.mean(),
            'avg_score_s_r_jagged': jagged_scores_s_r.mean(),
            'avg_score_p_nr_curved': curved_scores_p_nr.mean(),
            'avg_score_p_nr_jagged': jagged_scores_p_nr.mean()
        }

        return pd.DataFrame([analysis_results])

if __name__ == "__main__":
    # Initialize
    matcher = ImageTextMatcher(image_folder="images")
    image_paths = matcher.prepare_dataset_for_classification()
    classification_results = matcher.classify_dataset("llama", image_paths, possible_labels_s_r, possible_labels_p_nr)


    ## Change folder to [1,2,3]
    # Save results
    classification_results.to_csv("10/image_classifications.csv", index=False)
    
    # Save metrics
    classification_metrics = matcher.analyze_classification_results(classification_results)
    classification_metrics.to_csv("10/classification_metrics.csv", index=False)
    
    print("Classification complete. Results saved to CSV files.")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Classifying images 0-4:   0%|          | 0/4 [00:00<?, ?it/s]

batch number:  0
Predicted text: moolah.
Token probabilities: [0.7212, 1.0, 1.0, 1.0, 1.0]
Mean confidence: 0.9442


Classifying images 0-4:  25%|██▌       | 1/4 [00:33<01:39, 33.33s/it]

...

batch number:  32
Predicted text: loonah.
Token probabilities: [0.103, 1.0, 1.0, 1.0]
Mean confidence: 0.7757


Classifying images 32-34:  50%|█████     | 1/2 [00:28<00:28, 28.73s/it]

Predicted text: puhkuh.
Token probabilities: [0.1921, 1.0, 0.8299, 1.0, 1.0]
Mean confidence: 0.8044
batch number:  32
Predicted text: loonah.
Token probabilities: [0.1393, 1.0, 0.8135, 1.0]
Mean confidence: 0.7382


Classifying images 32-34: 100%|██████████| 2/2 [00:56<00:00, 28.16s/it]

Predicted text: tuhkuh.
Token probabilities: [0.5955, 0.4117, 0.7887, 1.0, 1.0]
Mean confidence: 0.7592
Classification complete. Results saved to CSV files.
